#Pre Requisites

#Load User_001.csv into DataFrame

In [0]:
df = spark.read.csv(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv",
    header=True,
    inferSchema=True,
)
df.display()

In [0]:
from pyspark.sql.functions import col
df.filter(col("country") == "India").display()
df.write.format("delta").save(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/output_delta/",
    mode = "overwrite"
)

# Transaction 01 - Write Data as Delta Format

In [0]:
df.write.format("delta").save(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/output_delta/",
    mode="overwrite"
)

#Reads the latest transaction

In [0]:
spark.read.load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/output_delta/"
).display()

#Maintains  Versions [Advantages]

In [0]:
spark.read.option("versionAsOf", 1).load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/output_delta/"
).display()

# Read the transaction log

## Approach 01


In [0]:
spark.read.load(
    format="text",
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/output_delta/_delta_log/00000000000000000000.json",
).display()

##Approach 02

In [0]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forPath( spark, "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/output_delta/")
delta_table.history().display()

#Transaction 03 - Updates are allowed

In [0]:
df_delta = spark.read.load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/output_delta/"
)
df_delta.createOrReplaceTempView("user_vw")

In [0]:
%sql
 
UPDATE user_vw
SET country='Bharat'
where country='India'